In [3]:
# Import statements

import sys

from time import sleep
 
import pygame

# Import statements from .py files

from settings import Settings

from game_stats import GameStats

from scoreboard import Scoreboard

from title import Title

from play_button import Play_button

from acknowledgements_button import Acknowledgements_button

from acknowledgements import Acknowledgements

from right_rectangle import RightRectangle

from left_rectangle import LeftRectangle

from ball import Ball

In [4]:
# Defining a Class:

class Pong:
    """Overall class to manage game assets and behavior."""
    
    def __init__(self):
        """Initialize the game, and create game resources."""

        pygame.init()
        self.settings = Settings()

        self.screen = pygame.display.set_mode((self.settings.screen_width, self.settings.screen_height))
        
        pygame.display.set_caption("Pong")
        
        # Ceate an instance to store game statistics, and create a scoreboard
        
        self.stats = GameStats(self)
        
        self.sb = Scoreboard(self)
        
        self.right_rectangle = RightRectangle(self)
        
        self.left_rectangle = LeftRectangle(self)
        
        self.ball = Ball(self)
        
        # Display the title
        
        self.title = Title(self, "Pong")
                
        # Make the buttons
        
        self.play_button = Play_button(self, "Play")
        
        # self.acknowledgements_button = Acknowledgements_button(self, "Acknowledgements")
        
        # Make the acknowledgements screen
        
        self.acknowledgements = Acknowledgements(self)
        
    def run_game(self):
        """Start the main loop for the game."""
        
        while True:
            
            self._check_events()
            
            if self.stats.game_active:
                
                self._out_of_bounds()
                # self._check_events_right_rectangle()
                # self._check_events_left_rectangle()
                self.right_rectangle.update()
                self.left_rectangle.update()
                self.ball.update()
                self._detect_ball_rectangle_collisions()
                # self.settings.increase_speed()
                
            self._update_screen()
            
    def _check_events(self):
        """Respond to keypresses and mouse events."""
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.display.quit()
                sys.exit()
                
            elif event.type == pygame.KEYDOWN:
                
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:
                
                self._check_keyup_events(event)
            
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                # self._check_acknowledgements_button(mouse_pos)
                self._check_play_button(mouse_pos)

            # elif event.type == pygame.MOUSEBUTTONDOWN:
                # mouse_pos = pygame.mouse.get_pos()
                # self._check_play_button(mouse_pos)

    # def _check_acknowledgements_button(self, mouse_pos):
        # """Display the acknowledgements screen when the play clicks 'acknowledgements'."""
        
        # button_clicked = self.acknowledgements_button.rect.collidepoint(mouse_pos)
        # if button_clicked and not self.stats.game_active:
            
            
            # Display the acknowledgemets
            # self.acknowledgements.blitme()

            # Hide the mouse cursor
            # pygame.mouse.set_visible(False)
            
    def _check_play_button(self, mouse_pos):
        """Start a new game when the player clicks 'play'."""
        
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and not self.stats.game_active:
            
            # Reset the game settings
            self.settings.initialize_dynamic_settings()
            
            # Reset the game statistics
            self.stats.reset_right_stats()
            self.stats.reset_left_stats()
            self.stats.game_active = True
            
            # Hide the mouse cursor
            pygame.mouse.set_visible(False)
            
    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        
        if event.key == pygame.K_UP:
            self.right_rectangle.right_rectangle_up = True

        elif event.key == pygame.K_DOWN:
            self.right_rectangle.right_rectangle_down = True

        elif event.key == pygame.K_w:
            self.left_rectangle.left_rectangle_up = True

        elif event.key == pygame.K_s:
            self.left_rectangle.left_rectangle_down = True

        elif event.key == pygame.K_q:
            pygame.display.quit()
            sys.exit()
                
    def _check_keyup_events(self, event):
        """Respond to key releases."""
        
        if event.key == pygame.K_UP:
            self.right_rectangle.right_rectangle_up = False

        elif event.key == pygame.K_DOWN:
            self.right_rectangle.right_rectangle_down = False

        elif event.key == pygame.K_w:
            self.left_rectangle.left_rectangle_up = False

        elif event.key == pygame.K_s:
            self.left_rectangle.left_rectangle_down = False
            
    def _detect_ball_rectangle_collisions(self):
        """Respond to ball-right_rectangle collisions."""
        
        collisions_right = pygame.Rect.colliderect(self.right_rectangle.rect, self.ball)
        
        collisions_left = pygame.Rect.colliderect(self.left_rectangle.rect, self.ball)
        
        if collisions_right:
            self.ball.update_after_collision_with_right_rectangle()
            self.settings.increase_speed()
            
        if collisions_left:
            self.ball.update_after_collision_with_left_rectangle()
            self.settings.increase_speed()
            
    def _decrement_right_rectangle_lives(self):
        """Respond to the ball going off the right side of the screen."""
        
        if self.stats.right_rectangles_left > 0:
            self.stats.right_score -= 1
            self.stats.right_rectangles_left -= 1
            # print("right")
            self.sb.prep_right_score()
        
            # Pause
            sleep(0.5)
            
        else:
            self.stats.game_active = False
    
    def _decrement_left_rectangle_lives(self):
        """Respond to the ball going off the left side of the screen."""
        
        if self.stats.left_rectangles_left > 0:
            self.stats.left_score -= 1
            self.stats.left_rectangles_left -= 1
            # print("left")
            self.sb.prep_left_score()
        
            # Pause
            sleep(0.5)
    
        else:
            self.stats.game_active = False
            
    def _out_of_bounds(self):
        """Respond to the ball going out of bounds."""
        
        if self.ball.x >= self.settings.screen_width:
            self._decrement_right_rectangle_lives()
            # self.sb.prep_right_score()
            self.ball.reset_ball()
            # self.ball.update()

            # Pause
            sleep(0.5)
            
        if self.ball.x <= 0:
            self._decrement_left_rectangle_lives()
            # self.sb.prep_left_score()
            self.ball.reset_ball()
            # self.ball.update()

            # Pause
            sleep(0.5)

    def _update_screen(self):
        """Update images on the screen, and flip to the new screen."""
        
        self.screen.fill(self.settings.bg_colour)

        self.right_rectangle.blitme()
        
        self.left_rectangle.blitme()
        
        self.ball.blitme()
        
        if not self.stats.game_active:
            self.acknowledgements.blitme()
        
        # Draw the title if the game is inative
        
        if not self.stats.game_active:
            self.title.draw_title()
        
        # Draw the score information
        
        self.sb.show_score()
        
        # Draw the play button if the game is inactive
        
        if not self.stats.game_active:
            self.play_button.draw_button()
            
         # Draw the acknowledgements button if the game is inactive
        
        # if not self.stats.game_active:
            # self.acknowledgements_button.draw_button()

        pygame.display.flip()
   
if __name__ == '__main__':
    # Make a game instance, and run the game
    pg = Pong()
    pg.run_game()

SystemExit: 

# Steps for Development

## Ball Movements Pt. 1 - Due Dec. 25

* Make the  ball move as planned. - <font color = 'red'> Done </font>


* Change the x and y speed values depending on which rectangle the ball hits. If the ball hits the left rectangle, increase the x value. If the ball hits the right rectangle, decrease the x value. - <font color = 'red'> Done </font>

## Ball Movements Pt. 2 - Due Jan. 2

* Change the x and y values depending on which edge of the screen the ball hits. If the ball hits the top of the screen, increase the y value. If the ball hits the bottom of the screen, decrease the y value. - <font color = 'red'> Done </font>


* Check to see if the ball hits the left edge of the screen or the right edge of the screen. If the ball hits the left edge of the screen, that means the left rectangle missed the ball. Then, the right hand side player gets a point. If the ball hits the right edge of the screen, that means the right rectangle missed the ball. Then, the left hand side player gets a point. - <font color = 'red'> Done </font>


## Scoring - Due Feb. 9

* Give each side a certain number of lives or misses. - <font color = 'red'> Done </font>


* Keep track of the score for each side. - <font color = 'red'> Done </font>


* Create an option to play the easy version of the game or the harder version.


* Increase the speed as the game goes by. - <font color = 'blue'> Later </font>


* Others?